In [1]:
import pandas as pd
import gensim.downloader as api
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from sklearn.decomposition import PCA
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emmahokken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
wv_google_300d = api.load('word2vec-google-news-300')
wv_twitter_50d = api.load('glove-twitter-50')


[==================================================] 100.0% 1662.8/1662.8MB downloaded
[==================================================] 100.0% 199.5/199.5MB downloaded


## Cosine similarity

In [1]:
from Vocab import Vocab
vocab = Vocab('data/vocab.csv', 3)
from get_predictions import get_predictions
from helpers import *

from ChainDataset import ChainDataset

chain_test_set = ChainDataset(
    data_dir='data/',
    segment_file='segments.json',
    chain_file='test_chains.json',
    vectors_file='vectors.json',
    split='test'
)


Initialising vocab from file.


In [4]:
models_dict = {'History':'model_history_blind_accs_2019-02-20-14-22-23.pkl',
            'No history': 'model_blind_accs_2019-02-17-21-18-7.pkl',
                'No image': 'model_history_noimg_accs_2019-03-01-14-25-34.pkl'}
# Also only uses the standard json files.
dataset_pred_no_hist = get_predictions(model_name='No history', models_dict=models_dict)
dataset_pred_hist = get_predictions(model_name='History', models_dict=models_dict)

loaded model: No history
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
params. normalize=True, mask=True, weight=5.5, weighting=True, batchsize=1, breaking=False
Dataparams. data_dir=./data, segmentfile=segments.json, vectorfile=vectors.json, chains_file=chains.json
processing test

Start prediction
predict no history
{'segment': [5, 212, 4, 166, 5, 12, 22, 782, 29, 234, 213, 10, 4, 416, 14, 12, 22, 7, 6, 49, 11, 6, 155, 295, 11, 6, 811, 16, 9, 95, 10, 5, 17, 8, 12], 'image_set': ['93469', '380128', '14238', '259745', '524866', '341060', '483794', '96757'], 'targets': [5], 'length': 35, 'preds': tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.]]]), 'loss': tensor(4.9204), 'ranks': [0]}
getting predicitons took 57.5736939907074 seconds
loaded model: History
Initialising vocab from file.
vocab len 3424
Initialising vocab from file.
Loaded seg2ranks and idlist
para

In [2]:
# Load in the segment ids that tell us which segment belongs to which index in the history dataset
segment_ids = get_seg_ids()

# Get inverted dict {seg_id:dataset_ind}
inv_list = create_inv_list(segment_ids)

# Create new history dataset with the segments in the same order as the no-history dataset
dataset_pred_hist_cp = reorder_datast(dataset_pred_hist, inv_list)

# Add the chain indices and the round in those chains per segement
dataset_pred_no_hist, dataset_pred_hist_cp = add_chains_rounds(dataset_pred_no_hist, dataset_pred_hist_cp, chain_test_set)

NameError: name 'dataset_pred_hist' is not defined

In [ ]:
dataframe = get_pred_dataframe(dataset_pred_no_hist, dataset_pred_hist_cp)
conditions_inds = get_conditions_inds(dataframe)


In [ ]:
get_condition_seg_hist(conditions_inds, dataset_pred_hist_cp)

In [3]:
history_df = pd.read_csv('history_predictions.csv')

incorrect_predictions = history_df.loc[history_df['correct'] == 0]
correct_predictions = history_df.loc[history_df['correct'] == 1]

stopword_list = stopwords.words('english')

stopword_list.append("-B-")
stopword_list.append("-A-")
stopword_list.append("<unk>")

stemmer = nltk.stem.porter.PorterStemmer()

incorrect_predictions['decoded_segment']

9       ['-B-', 'okay', '.', 'i', "'m", 'done', 'anymo...
23      ['-B-', 'i', 'have', 'the', 'same', 'picture',...
60      ['-B-', 'guy', 'on', 'white', 'bike', '-A-', '...
64      ['-B-', 'do', 'you', 'have', 'the', 'lady', 's...
71      ['-B-', 'kk', 'little', 'girl', 'helping', 'da...
80      ['-B-', 'cool', 'common', 'ready', 'when', 'yo...
110     ['-B-', 'i', 'have', 'a', 'person', 'pushing',...
146     ['-B-', 'do', 'you', 'have', '2', 'people', 'r...
159     ['-A-', 'i', 'have', 'that', 'also', '.', 'on'...
176     ['-A-', 'my', 'second', 'one', 'is', 'a', 'hot...
177     ['-A-', 'my', 'first', 'one', 'is', 'a', 'hot'...
180     ['-A-', 'do', 'they', 'have', 'brown', 'hair',...
185     ['-B-', 'and', 'do', 'you', 'have', 'stripe', ...
187     ['-A-', 'i', 'have', 'a', 'woman', 'in', 'a', ...
189     ['-A-', 'i', 'have', 'two', 'kids', 'carrying'...
214     ['-B-', 'laptop', 'with', 'meal', 'and', 'drin...
217     ['-B-', 'no', 'i', 'do', "n't", 'have', 'that'...
219     ['-A-'

In [4]:
vectors = []

# incorrect_predictions['decoded_segment'], decoded

wrong_pred = incorrect_predictions['decoded_segment']
good_pred = correct_predictions['decoded_segment']

all_predictions = [good_pred, wrong_pred]


for prediction in all_predictions:

    vector = 0
    i = 0
    
    for sentence in prediction:
        for word in sentence.split(","):

            word = word.replace("'", "")
            word = word.replace(" ", "")
            word = word.lower()

    #         if word is stopword, skip the word
            if word in stopword_list:
                continue

            word = stemmer.stem(word)

            try:

                vector += wv_twitter_50d[word]
                i += 1

            except:
                None
                
    vector = vector / i
    vectors.append(vector)


print(vectors)


[array([-8.92508179e-02, -2.45265096e-01,  8.19780305e-02, -3.33554409e-02,
       -8.68604779e-02, -8.33939482e-03,  5.46882093e-01,  1.65179178e-01,
        2.08532125e-01,  3.48601133e-01, -9.33928862e-02,  1.79920807e-01,
       -3.28285956e+00,  2.69561689e-02, -1.22719491e-02,  1.84684321e-01,
        9.91539881e-02, -3.34496498e-01, -1.51952639e-01, -1.23826720e-01,
       -2.20019653e-01,  5.86802664e-04,  1.69557005e-01, -1.01809360e-01,
        1.09491479e-02,  2.17816290e-02, -1.87915027e-01, -6.05420507e-02,
        8.28245729e-02, -1.98522657e-01,  4.09429595e-02,  3.18477005e-01,
        6.46469221e-02, -3.03640574e-01,  3.02216321e-01, -2.25010961e-02,
       -9.80426744e-03,  2.73617413e-02,  6.34614006e-02, -1.49778475e-03,
       -8.75186801e-01,  9.57668126e-02, -1.70625001e-01,  1.03947945e-01,
        2.70428836e-01, -1.31704006e-02,  3.16535622e-01,  3.77877802e-02,
       -8.66262801e-03,  2.58087534e-02], dtype=float32), array([-7.83506781e-02, -2.46343359e-01, 

In [5]:
a = vectors[0].reshape(-1, 1)
b = vectors[1].reshape(-1, 1)
# print(a.shape)
a = a.mean()
b = b.mean()

# print(type(a))

# a = np.array([1, 4, 5, 765432, 5, 3])
# a = a.reshape(-1, 1)

pca = PCA(n_components=0.50, whiten = True)
result = pca.fit_transform(a)
result

# result
# pca = PCA(n_components=2)
# # pca.fit(vectors)
# # PCA(n_components=2)


# result = pca.fit(vectors)

# pca.explained_variance_ratio

ValueError: Expected 2D array, got scalar array instead:
array=-0.056815993040800095.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [251]:
a = [1, 2, 20000]
b = [4, 2, 5434543]

pca = PCA(n_components=2)
pca.fit_transform(a,b )

ValueError: Expected 2D array, got 1D array instead:
array=[1.e+00 2.e+00 2.e+04].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

## POS tags distribution


In [72]:
from collections import Counter
import nltk
import pickle
from nltk import word_tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('universal_tagset')

with open('oov_dictionary.pickle', 'rb') as f:
    oov_dict = pickle.load(f)

top_10 = ['NOUN', 'DET', 'VERB', 'ADP', 'ADJ', '.', 'ADV', 'NUM', 'PRON', 'CONJ']
top_4 = ['NOUN', 'VERB', 'ADJ', 'ADV']

''' explanation:

ADJ	adjective	new, good, high, special, big, local
ADP	adposition	on, of, at, with, by, into, under
ADV	adverb	really, already, still, early, now
CONJ	conjunction	and, or, but, if, while, although
DET	determiner, article	the, a, some, most, every, no, which
NOUN	noun	year, home, costs, time, Africa
NUM	numeral	twenty-four, fourth, 1991, 14:24
PRT	particle	at, on, out, over per, that, up, with
PRON	pronoun	he, their, her, its, my, I, us
VERB	verb	is, say, told, given, playing, would
.	punctuation marks	. , ; !
X	other	ersatz, esprit, dunno, gr8, univeristy

'''

oov_dict



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\lolab\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


{'grey': 'gray',
 'ok.': 'OK',
 't-shirt': 'tshirt',
 'w/': 'with',
 'frige': 'fridge',
 'red/white': 'red and white',
 'doughnuts': 'donuts',
 'didnt': "didn't",
 'ok..': 'ok',
 'dr.': 'dr',
 'no..': 'no',
 'doughnut': 'donut',
 'yes..': 'yes',
 'yellow/orange': 'yellow and orange',
 'pic..': 'picture',
 'umberlla': 'umbrella',
 'cest': 'chest',
 'k..next': 'ok, next',
 'surboard': 'surfboard',
 'blue/white': 'blue and white',
 'man/woman': 'man or woman',
 'doesnt': "doesn't",
 'scheppa': 'schepps',
 'guy/girl': 'guy or girl',
 'blue/green': 'blue and green',
 'schepp': 'schepps',
 'blue/black': 'blue and black',
 'motarcycle': 'motorcycle',
 'red/black': 'red and black',
 'red/white/blue': 'red, white and blue',
 'colour': 'color',
 'close-up': 'closeup',
 'green/yellow': 'green and yellow',
 'red/pink': 'red and pink',
 'buddist': 'buddhist',
 'longsleeves': 'long sleeves',
 'ihave': 'i have',
 'yellow/green': 'yellow and green',
 'no-': 'no',
 'k.': 'ok',
 'surfobard': 'surfboard'

In [75]:
# text = " Guru99 is one of the best sites to learn WEB, SAP, Ethical Hacking and much more online."
messages = ['ik ben bob en daar is het strand maar toen zei hij']
pos_tags = []

for message in messages:   
    m_tokens = [i for i in word_tokenize(message.lower())]
    
    
    m_tokens_spell = []
    for word in m_tokens:
        if word not in oov_dict:
            m_tokens_spell.append(word)
        else:
            m_tokens_spell.append(oov_dict[word])
            
    pos_tags += [item[1] for item in pos_tag(m_tokens_spell, target='universal')]

# if len(pos_tags) == 0:
#     continue
pos_tag_dict = dict()       
for key,value in dict(Counter(pos_tags)).items():
    pos_tag_dict[key] = [value]  
    

# pos_tag_dict['Round_Nr'] = round_nr
# round_entry_df = pd.DataFrame.from_dict(pos_tag_dict)
# pos_statistics.append(round_entry_df)

TypeError: pos_tag() got an unexpected keyword argument 'target'